# PROJET ENERGIE - TEST STATISTIQUE
**Objectifs :** Au moins 5 représentations graphiques construites à partir de votre jeu de données, visuelles et surtout pertinentes. Il faudra aussi accompagner votre travail d'analyses statistiques de qualité. Pour chaque visualisation, j’attends : Un commentaire précis, qui analyse la figure et apporte un avis “métier”. Une validation du constat par des manipulations de données, ou un test statistique.

**Préparation du jeu de données :**

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
df = pd.read_csv('gdrive/My Drive/Colab Notebooks/ProjetEnergie/eco2mix-regional-cons-def.csv', sep=';')

# suppression colonne
df = df.drop(columns = 'Column 30')

# suppression 12 premières lignes
import numpy as np
variables_numeriques = df.select_dtypes(include=['float']).columns
df = df.dropna(axis=0, how='all', subset=variables_numeriques)

# suppression des valeurs manquantes pour les colonnes 'Nucléaire (MW)' et 'Eolien (MW)
df['Nucléaire (MW)'] = df['Nucléaire (MW)'].fillna(0)
df['Eolien (MW)'] = df['Eolien (MW)'].fillna(0)

df.head()

Mounted at /content/gdrive


<ipython-input-2-00bdd74607cb>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Nucléaire (MW)'] = df['Nucléaire (MW)'].fillna(0)


,Code INSEE région,Région,Nature,Date,Heure,Date - Heure,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),...,TCO Nucléaire (%),TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%)
12,52,Pays de la Loire,Données définitives,2013-01-01,00:30,2013-01-01T00:30:00+01:00,3595.0,127.0,0.0,182.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,24,Centre-Val de Loire,Données définitives,2013-01-01,00:30,2013-01-01T00:30:00+01:00,2476.0,90.0,9085.0,508.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,44,Grand Est,Données définitives,2013-01-01,00:30,2013-01-01T00:30:00+01:00,4943.0,319.0,9137.0,1109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,11,Île-de-France,Données définitives,2013-01-01,00:30,2013-01-01T00:30:00+01:00,9134.0,685.0,0.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,27,Bourgogne-Franche-Comté,Données définitives,2013-01-01,00:30,2013-01-01T00:30:00+01:00,2357.0,236.0,0.0,106.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df['Mois'] = df['Date'].astype(str).str[5:-3]
energie_mois = df.groupby('Mois').agg({'Consommation (MW)' : 'mean', 'Thermique (MW)' : 'mean', 'Nucléaire (MW)' : 'mean',
                                                 'Eolien (MW)' : 'mean', 'Solaire (MW)' : 'mean', 'Hydraulique (MW)' : 'mean',
                                                 'Bioénergies (MW)' : 'mean'})
energie_mois = energie_mois.reset_index()

**Premier test statistique :**

In [4]:
#Choix des hypothèses : Nous souhaitons savoir s'il existe une corrélation entre la variable date (mois) et la variable consommation, le but étant de savoir si le consommation
#est différente selon le mois de l'année. Donc 2 hypothèses se dégagent:
print("Les hypothèses sont : ")
print("H0 : Le mois n'influence pas la consommation d'énergie")
print("H1 : Le mois influence la consommation d'énergie")
#Le choix du test : Nous allons choisir le test Anova puique nous devons étudier une intéraction entre une variable catégorielle et une variable continue.
# Mise en application du test : Test Anova

Les hypothèses sont : 
H0 : Le mois n'influence pas la consommation d'énergie
H1 : Le mois influence la consommation d'énergie


In [5]:
from scipy import stats
import statsmodels.api as sm
result = stats.f_oneway(energie_mois['Consommation (MW)'], energie_mois['Mois'])
if result.pvalue > 0.05:
    print("H0 : Le mois n'influence pas la consommation d'énergie")
else:
    print("H1 : Le mois influence la consommation d'énergie")

print("pvalue : ", result.pvalue)

H1 : Le mois influence la consommation d'énergie
pvalue :  4.311662907431899e-15


**Second test statistique :**

In [ ]:
#Choix des hypothèses : Nous souhaitons savoir s'il existe une corrélation entre la variable région et la variable production, le but étant de savoir si la production est
#différente selon une localisation en France, la région. Donc 2 hypothèses se dégagent:
print("Les hypothèses sont : ")
print("H0 : La région n'influence pas la production d'énergie")
print("H1 : La région influence la production d'énergie")
#Le choix du test : Nous allons choisir le test Anova puique nous devons étudier une intéraction entre une variable catégorielle et une variable continue.
# Mise en application du test : Test Anova

Les hypothèses sont : 
H0 : La région n'influence pas la production d'énergie
H1 : La région influence la production d'énergie


In [ ]:
df['sum_prod'] = df['Thermique (MW)'] + df['Nucléaire (MW)'] + df['Eolien (MW)'] + df['Solaire (MW)'] + df['Hydraulique (MW)'] + df['Bioénergies (MW)']
energie_region = df.groupby('Code INSEE région').agg({'Consommation (MW)' : 'mean', 'sum_prod' : 'mean'})
energie_region = energie_region.reset_index()

from scipy import stats
import statsmodels.api as sm
result = stats.f_oneway(energie_region['sum_prod'], energie_region['Code INSEE région'])
if result.pvalue > 0.05:
    print("H0 : La région n'influence pas la production d'énergie")
else:
    print("H1 : La région influence la production d'énergie")

print("pvalue : ", result.pvalue)

H1 : La région influence la production d'énergie
pvalue :  0.0008522665859492165
